In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('base_clientes.xlsx')

In [3]:
df.head()

,Número do Cliente,MCC,Data Apuração,Data de Entrada do Cliente,Canal,Cidade,Volume Transacionado,Quantidade de Transações,Net MDR (Receita),Aluguel (Receita),Pre-pagamento (Receita),Total (Receita),Pre-pagamento (Custos),Equipamento/Logística (Custos),Comissões (Custos),Transacionais (Custos),Outros (Custos),Total (Custos),Lucro,grupos
0,100910138884288,5309,2016-10-01,2016-01-07,Canal 3,Niterói,189012.41,2235.0,122.27,74.09,0.00,196.36,0.00,-141.11,0.0,-143.14,-46.39,-330.64,-134.28,1
1,100101693498007,5963,2016-10-01,2015-12-10,Canal 3,Rio de Janeiro,0.00,0.0,0.00,0.00,0.00,0.00,0.00,-13.00,0.0,0.00,0.00,-13.00,-13.00,1
2,100990966870135,5814,2016-10-01,2016-03-29,Canal 3,Rio de Janeiro,240922.64,1190.0,1149.34,143.38,4317.06,5609.78,-1525.63,-312.13,0.0,-108.58,-146.24,-2092.58,3517.21,1
3,100112746462403,7230,2016-10-01,2016-07-26,Canal 3,Niterói,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,-24.01,-24.01,-24.01,1
4,100101114264588,7997,2016-10-01,2016-04-13,Canal 1,Brasília,3634.70,56.0,61.73,0.00,0.00,61.73,0.00,0.00,0.0,-12.58,-52.55,-65.13,-3.39,2


In [4]:
df['Status do Cliente'] = df['Quantidade de Transações'].apply(lambda x: 'Ativo' if x>0 else 'Inativo')

In [5]:
data_ref = df['Data Apuração'].unique()

In [7]:
for i in range(len(data_ref)):
    
    if data_ref[i] > min(df['Data Apuração']):
        
        lista_inativos_mes_atual = df.loc[ 
            (df['Data Apuração']==data_ref[i]) & 
            (df['Status do Cliente']=='Inativo'), 
            'Número do Cliente' ]

        numerador = len(
            df.loc[ 
            (df['Data Apuração']==data_ref[i-1]) & 
            (df['Status do Cliente']=='Ativo') &
            (df['Número do Cliente'].isin(list(lista_inativos_mes_atual))), 
            'Número do Cliente' 
            ]
        )
        
        denominador = len(
            df.loc[ 
            (df['Data Apuração']==data_ref[i-1]) & 
            (df['Status do Cliente']=='Ativo'), 
            'Número do Cliente' 
            ]
        )

        churn = round(numerador/denominador, 4)
        tempo_medio = round(1/churn, 1)

        print( 'Clientes inativos no mês atual:', len(lista_inativos_mes_atual) )
        print( 'Clientes ativos no mês passado', numerador)
        print( 'Total de clientes ativos no mês passado:', denominador )
        print( 'Churn para a data de referência', data_ref[i], 'é de', churn )
        print( 'O tempo de vida médio para a data de refência é de', tempo_medio, 'meses' )
        print( '------------------------------------------------------------------------------\n' )

    else:

        print( 'Não é possível determinar o churn e o tempo médio de vida para a \ndata de', data_ref[i], 'pois não há histórico de dados anteriores!' )
        print( '------------------------------------------------------------------------------\n' )
        churn = 0
        tempo_medio = 0

    df.loc[ (df['Data Apuração']==data_ref[i]), 'Churn' ] = churn
    df.loc[ (df['Data Apuração']==data_ref[i]), 'Tempo de Vida Médio (Meses)' ] = tempo_medio
        

Não é possível determinar o churn e o tempo médio de vida para a 
data de 2016-10-01T00:00:00.000000000 pois não há histórico de dados anteriores!
------------------------------------------------------------------------------

Clientes inativos no mês atual: 549
Clientes ativos no mês passado 105
Total de clientes ativos no mês passado: 1719
Churn para a data de referência 2016-11-01T00:00:00.000000000 é de 0.0611
O tempo de vida médio para a data de refência é de 16.4 meses
------------------------------------------------------------------------------

Clientes inativos no mês atual: 524
Clientes ativos no mês passado 108
Total de clientes ativos no mês passado: 1877
Churn para a data de referência 2016-12-01T00:00:00.000000000 é de 0.0575
O tempo de vida médio para a data de refência é de 17.4 meses
------------------------------------------------------------------------------

Clientes inativos no mês atual: 565
Clientes ativos no mês passado 138
Total de clientes ativos no mês pass

In [10]:
df.to_excel('dados_consolidados.xlsx', index=False)